In [ ]:
#IMPORTIMET

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

In [ ]:
#WEB SCRAPING KODI

In [7]:
url = "https://asp.gov.al/"  # Website
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser') #perdorim beautiful soup

data = []

# Ekstraktimi i vendodhjes marre nga titujt
def extract_location_from_title(title):
    cities = ["Tiranë", "Durrës", "Fier", "Korçë", "Elbasan", "Gjirokastër", "Has", "Shkodër", "Berat"]
    normalized_title = title.lower()
    for city in cities:
        if city.lower() in normalized_title:
            return city
    return "Vendodhja nuk u gjet"

# OpenWeather API
api_url = "https://api.openweathermap.org/data/2.5/weather"
api_key = "c1c6ef3ed03009165d3cbd1f76b7cb4a"  # Kodi API per weatherin

# Ben ekstrakt titujve vendodhjen dhe motin
for article in soup.find_all('h2'):  
    title = article.get_text(strip=True)
    location = extract_location_from_title(title)
    
    weather_info = "Vendodhja nuk ekziston"
    if location != "Vendodhja nuk u gjet":
        params = {
            "q": location,
            "appid": api_key,
            "units": "metric"  # Marrim temp n C
        }
        weather_response = requests.get(api_url, params=params).json()
        if "main" in weather_response:
            weather_info = {
                "temperature": weather_response["main"]["temp"],
                "description": weather_response["weather"][0]["description"]
            }
        else:
            weather_info = "Nuk u gjeten te dhena per motin"
    
    data.append({"title": title, "location": location, "weather": weather_info})

In [8]:
#Ruajta ne JSON

In [9]:
json_filename = "ASP_tedhena.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [10]:
#Ruajtja ne CSV EXCEL

In [11]:
csv_filename = "ASP_tedhena.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["title", "location", "weather"])
    writer.writeheader()
    writer.writerows(data)

In [12]:
#ENKRIPTIMI AES

In [13]:
key = os.urandom(32)  # Gjeneron nje celes random 256 bit
iv = os.urandom(16)   # Generon IV

cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
encryptor = cipher.encryptor()

with open(json_filename, 'rb') as f:
    plaintext = f.read()

ciphertext = encryptor.update(plaintext) + encryptor.finalize()

encrypted_filename = "enkriptimi_tedhenave.aes"
with open(encrypted_filename, 'wb') as f:
    f.write(iv + ciphertext)  # Pergatit IV for dekriptim

In [14]:
#ENKRIPTIM RSA 

In [15]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

encrypted_key = public_key.encrypt(
    key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

key_filename = "celesi_enkriptimit.pem"
with open(key_filename, 'wb') as f:
    f.write(encrypted_key)

print(f"Te dhenat u moren, u enkriptuan dhe u ruajten {json_filename}, {csv_filename}, and {encrypted_filename}.")

Te dhenat u moren, u enkriptuan dhe u ruajten ASP_tedhena.json, ASP_tedhena.csv, and enkriptimi_tedhenave.aes.
